# Spark + Airflow + MLflow: Orquestación de Pipelines ML

## Objetivos
- Crear un pipeline de datos con Apache Spark
- Orquestar el pipeline con Apache Airflow
- Trackear experimentos y métricas con MLflow
- Implementar un flujo end-to-end de ML

## Conceptos Clave
- **Spark**: Motor de procesamiento distribuido
- **Airflow**: Plataforma de orquestación de workflows
- **MLflow**: Plataforma de gestión del ciclo de vida ML

## Arquitectura del Pipeline
```
Airflow DAG
    ↓
1. Ingesta de datos (Spark)
    ↓
2. Transformación (Spark)
    ↓
3. Feature Engineering (Spark)
    ↓
4. Entrenamiento de modelo (Spark ML + MLflow)
    ↓
5. Evaluación y registro (MLflow)
```

## 1. Setup e Importaciones

In [ ]:
# Importaciones necesarias
import os
import sys
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Spark
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, when, mean, stddev, count
from pyspark.sql.types import StructType, StructField, StringType, DoubleType, IntegerType

# Spark ML
from pyspark.ml.feature import VectorAssembler, StandardScaler, StringIndexer
from pyspark.ml.classification import RandomForestClassifier, LogisticRegression
from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator
from pyspark.ml import Pipeline

# MLflow
import mlflow
import mlflow.spark
from mlflow.tracking import MlflowClient

# Airflow (para definir DAGs)
from airflow import DAG
from airflow.operators.python import PythonOperator
from airflow.providers.apache.spark.operators.spark_submit import SparkSubmitOperator

# Utilidades
import pandas as pd
import numpy as np

print("✓ Importaciones completadas")

## 2. Configuración de Spark y MLflow

In [ ]:
# Crear SparkSession
spark = SparkSession.builder \
    .appName("Airflow-MLflow-Pipeline") \
    .master("local[*]") \
    .config("spark.driver.memory", "4g") \
    .config("spark.executor.memory", "4g") \
    .config("spark.sql.shuffle.partitions", "4") \
    .getOrCreate()

# Configurar MLflow
mlflow.set_tracking_uri("http://localhost:5000")  # Cambiar si usas servidor remoto
mlflow.set_experiment("spark-airflow-pipeline")

print(f"✓ Spark version: {spark.version}")
print(f"✓ MLflow tracking URI: {mlflow.get_tracking_uri()}")
print(f"✓ MLflow experiment: {mlflow.get_experiment_by_name('spark-airflow-pipeline').name}")

## 3. Simulación de DAG de Airflow

En un entorno real, este DAG estaría en `airflow/dags/`. Aquí lo simulamos para propósitos educativos.

In [ ]:
# Definición del DAG (esto normalmente va en airflow/dags/spark_ml_pipeline.py)

default_args = {
    'owner': 'yusef',
    'depends_on_past': False,
    'start_date': datetime(2024, 1, 1),
    'email_on_failure': False,
    'email_on_retry': False,
    'retries': 1,
    'retry_delay': timedelta(minutes=5),
}

# Estructura del DAG
dag_structure = """
DAG: spark_ml_pipeline
├── Task 1: ingest_data
├── Task 2: transform_data
├── Task 3: feature_engineering
├── Task 4: train_model
└── Task 5: evaluate_model
"""

print(dag_structure)
print("\nNota: En producción, este DAG se ejecutaría en Airflow")
print("Aquí ejecutaremos cada tarea manualmente para demostración")

## 4. Task 1: Ingesta de Datos

Generamos datos sintéticos que simulan transacciones de clientes para detectar fraude.

In [ ]:
def ingest_data():
    """Task 1: Ingesta de datos usando Spark"""
    print("[Task 1] Iniciando ingesta de datos...")
    
    # Generar datos sintéticos
    np.random.seed(42)
    n_samples = 10000
    
    data = {
        'transaction_id': range(n_samples),
        'amount': np.random.exponential(100, n_samples),
        'merchant_category': np.random.choice(['retail', 'online', 'restaurant', 'travel'], n_samples),
        'transaction_hour': np.random.randint(0, 24, n_samples),
        'customer_age': np.random.randint(18, 80, n_samples),
        'is_international': np.random.choice([0, 1], n_samples, p=[0.9, 0.1]),
        'num_previous_transactions': np.random.randint(0, 100, n_samples),
        'is_fraud': np.random.choice([0, 1], n_samples, p=[0.95, 0.05])  # 5% fraude
    }
    
    # Crear DataFrame de Spark
    df = spark.createDataFrame(pd.DataFrame(data))
    
    print(f"✓ Datos ingestados: {df.count()} registros")
    print(f"✓ Columnas: {df.columns}")
    
    return df

# Ejecutar Task 1
raw_df = ingest_data()
raw_df.show(5)
raw_df.printSchema()

## 5. Task 2: Transformación de Datos

In [ ]:
def transform_data(df):
    """Task 2: Transformación y limpieza de datos"""
    print("[Task 2] Iniciando transformación de datos...")
    
    # Limpieza: eliminar duplicados
    df_clean = df.dropDuplicates(['transaction_id'])
    
    # Crear nuevas features
    df_transformed = df_clean \
        .withColumn('amount_log', col('amount').log1p()) \
        .withColumn('is_night_transaction', when(col('transaction_hour') < 6, 1).otherwise(0)) \
        .withColumn('is_large_amount', when(col('amount') > 500, 1).otherwise(0)) \
        .withColumn('risk_score', 
                   col('is_international') * 0.3 + 
                   col('is_night_transaction') * 0.2 + 
                   col('is_large_amount') * 0.5)
    
    # Estadísticas
    stats = df_transformed.select(
        count('*').alias('total_records'),
        mean('amount').alias('avg_amount'),
        stddev('amount').alias('stddev_amount')
    ).collect()[0]
    
    print(f"✓ Registros transformados: {stats['total_records']}")
    print(f"✓ Monto promedio: ${stats['avg_amount']:.2f}")
    print(f"✓ Desviación estándar: ${stats['stddev_amount']:.2f}")
    
    return df_transformed

# Ejecutar Task 2
transformed_df = transform_data(raw_df)
transformed_df.select('transaction_id', 'amount', 'amount_log', 'risk_score', 'is_fraud').show(5)

## 6. Task 3: Feature Engineering

In [ ]:
def feature_engineering(df):
    """Task 3: Preparación de features para ML"""
    print("[Task 3] Iniciando feature engineering...")
    
    # Indexar variables categóricas
    indexer = StringIndexer(inputCol='merchant_category', outputCol='merchant_category_idx')
    df_indexed = indexer.fit(df).transform(df)
    
    # Seleccionar features para el modelo
    feature_cols = [
        'amount_log',
        'merchant_category_idx',
        'transaction_hour',
        'customer_age',
        'is_international',
        'num_previous_transactions',
        'is_night_transaction',
        'is_large_amount',
        'risk_score'
    ]
    
    # Ensamblar features en vector
    assembler = VectorAssembler(inputCols=feature_cols, outputCol='features_raw')
    df_assembled = assembler.transform(df_indexed)
    
    # Escalar features
    scaler = StandardScaler(inputCol='features_raw', outputCol='features', withMean=True, withStd=True)
    scaler_model = scaler.fit(df_assembled)
    df_final = scaler_model.transform(df_assembled)
    
    # Seleccionar columnas finales
    df_ml = df_final.select('features', col('is_fraud').alias('label'))
    
    print(f"✓ Features creadas: {len(feature_cols)}")
    print(f"✓ Features: {feature_cols}")
    
    return df_ml, feature_cols

# Ejecutar Task 3
ml_df, feature_names = feature_engineering(transformed_df)
ml_df.show(5, truncate=False)

## 7. Task 4: Entrenamiento de Modelo con MLflow

In [ ]:
def train_model(df, feature_names):
    """Task 4: Entrenamiento de modelo con tracking en MLflow"""
    print("[Task 4] Iniciando entrenamiento de modelo...")
    
    # Split train/test
    train_df, test_df = df.randomSplit([0.8, 0.2], seed=42)
    
    print(f"✓ Training set: {train_df.count()} registros")
    print(f"✓ Test set: {test_df.count()} registros")
    
    # Iniciar MLflow run
    with mlflow.start_run(run_name="random-forest-fraud-detection") as run:
        
        # Log parámetros
        params = {
            'model_type': 'RandomForest',
            'num_trees': 100,
            'max_depth': 10,
            'feature_subset_strategy': 'auto'
        }
        mlflow.log_params(params)
        
        # Entrenar modelo
        rf = RandomForestClassifier(
            featuresCol='features',
            labelCol='label',
            numTrees=params['num_trees'],
            maxDepth=params['max_depth'],
            featureSubsetStrategy=params['feature_subset_strategy'],
            seed=42
        )
        
        print("\n⏳ Entrenando Random Forest...")
        model = rf.fit(train_df)
        print("✓ Modelo entrenado")
        
        # Feature importance
        feature_importance = list(zip(feature_names, model.featureImportances.toArray()))
        feature_importance.sort(key=lambda x: x[1], reverse=True)
        
        print("\n📊 Top 5 Features más importantes:")
        for feat, importance in feature_importance[:5]:
            print(f"  - {feat}: {importance:.4f}")
            mlflow.log_metric(f"feature_importance_{feat}", importance)
        
        # Guardar modelo
        mlflow.spark.log_model(model, "random-forest-model")
        
        # Log artifacts adicionales
        mlflow.log_param("num_features", len(feature_names))
        mlflow.set_tag("pipeline_stage", "training")
        mlflow.set_tag("framework", "spark-ml")
        
        print(f"\n✓ Run ID: {run.info.run_id}")
        
        return model, train_df, test_df, run.info.run_id

# Ejecutar Task 4
model, train_df, test_df, run_id = train_model(ml_df, feature_names)

## 8. Task 5: Evaluación del Modelo

In [ ]:
def evaluate_model(model, test_df, run_id):
    """Task 5: Evaluación del modelo y logging de métricas"""
    print("[Task 5] Iniciando evaluación del modelo...")
    
    # Predicciones
    predictions = model.transform(test_df)
    
    # Continuar el run anterior
    with mlflow.start_run(run_id=run_id):
        
        # Evaluadores
        binary_evaluator = BinaryClassificationEvaluator(
            labelCol='label',
            rawPredictionCol='rawPrediction',
            metricName='areaUnderROC'
        )
        
        multiclass_evaluator = MulticlassClassificationEvaluator(
            labelCol='label',
            predictionCol='prediction'
        )
        
        # Calcular métricas
        auc = binary_evaluator.evaluate(predictions)
        accuracy = multiclass_evaluator.evaluate(predictions, {multiclass_evaluator.metricName: "accuracy"})
        precision = multiclass_evaluator.evaluate(predictions, {multiclass_evaluator.metricName: "weightedPrecision"})
        recall = multiclass_evaluator.evaluate(predictions, {multiclass_evaluator.metricName: "weightedRecall"})
        f1 = multiclass_evaluator.evaluate(predictions, {multiclass_evaluator.metricName: "f1"})
        
        # Log métricas
        metrics = {
            'test_auc': auc,
            'test_accuracy': accuracy,
            'test_precision': precision,
            'test_recall': recall,
            'test_f1': f1
        }
        
        mlflow.log_metrics(metrics)
        
        # Mostrar resultados
        print("\n📊 Métricas del Modelo:")
        print(f"  • AUC-ROC: {auc:.4f}")
        print(f"  • Accuracy: {accuracy:.4f}")
        print(f"  • Precision: {precision:.4f}")
        print(f"  • Recall: {recall:.4f}")
        print(f"  • F1-Score: {f1:.4f}")
        
        # Matriz de confusión (simplificada)
        confusion = predictions.groupBy('label', 'prediction').count().toPandas()
        print("\n📊 Matriz de Confusión:")
        print(confusion)
        
        # Ejemplos de predicciones
        print("\n🔍 Ejemplos de Predicciones:")
        predictions.select('label', 'prediction', 'probability').show(10, truncate=False)
        
        return metrics

# Ejecutar Task 5
metrics = evaluate_model(model, test_df, run_id)

## 9. Resumen del Pipeline

In [ ]:
print("="*70)
print("RESUMEN DEL PIPELINE SPARK + AIRFLOW + MLFLOW")
print("="*70)

print("\n✅ Tasks Completadas:")
print("  1. ✓ Ingesta de datos (Spark)")
print("  2. ✓ Transformación de datos (Spark)")
print("  3. ✓ Feature Engineering (Spark ML)")
print("  4. ✓ Entrenamiento de modelo (Random Forest)")
print("  5. ✓ Evaluación y registro (MLflow)")

print("\n📊 Métricas Finales:")
for metric, value in metrics.items():
    print(f"  • {metric}: {value:.4f}")

print("\n🔗 MLflow Tracking:")
print(f"  • Run ID: {run_id}")
print(f"  • Tracking URI: {mlflow.get_tracking_uri()}")
print(f"  • Experiment: spark-airflow-pipeline")

print("\n💡 Próximos Pasos:")
print("  1. Implementar el DAG completo en Airflow")
print("  2. Configurar scheduling automático (daily, hourly, etc.)")
print("  3. Añadir alertas y monitoring")
print("  4. Deploy del modelo en producción")
print("  5. Implementar retraining automático")

print("\n" + "="*70)

## 10. Archivo DAG para Airflow (Código de Producción)

Este código debería ir en `airflow/dags/spark_ml_pipeline.py`:

In [ ]:
# Código de ejemplo para airflow/dags/spark_ml_pipeline.py

dag_code = '''
from airflow import DAG
from airflow.operators.python import PythonOperator
from airflow.providers.apache.spark.operators.spark_submit import SparkSubmitOperator
from datetime import datetime, timedelta

default_args = {
    'owner': 'yusef',
    'depends_on_past': False,
    'start_date': datetime(2024, 1, 1),
    'email': ['yusef@example.com'],
    'email_on_failure': True,
    'email_on_retry': False,
    'retries': 2,
    'retry_delay': timedelta(minutes=5),
}

dag = DAG(
    'spark_ml_pipeline',
    default_args=default_args,
    description='Pipeline ML con Spark y MLflow',
    schedule_interval='@daily',  # Ejecutar diariamente
    catchup=False,
    tags=['spark', 'ml', 'mlflow'],
)

# Task 1: Ingesta
ingest_task = SparkSubmitOperator(
    task_id='ingest_data',
    application='scripts/ingest_data.py',
    conn_id='spark_default',
    dag=dag,
)

# Task 2: Transformación
transform_task = SparkSubmitOperator(
    task_id='transform_data',
    application='scripts/transform_data.py',
    conn_id='spark_default',
    dag=dag,
)

# Task 3: Feature Engineering
feature_task = SparkSubmitOperator(
    task_id='feature_engineering',
    application='scripts/feature_engineering.py',
    conn_id='spark_default',
    dag=dag,
)

# Task 4: Training
train_task = SparkSubmitOperator(
    task_id='train_model',
    application='scripts/train_model.py',
    conn_id='spark_default',
    dag=dag,
)

# Task 5: Evaluation
eval_task = PythonOperator(
    task_id='evaluate_model',
    python_callable=evaluate_model_func,
    dag=dag,
)

# Definir dependencias
ingest_task >> transform_task >> feature_task >> train_task >> eval_task
'''

print("Código del DAG para Airflow:")
print(dag_code)

# Guardar a archivo
with open('../scripts/example_airflow_dag.py', 'w') as f:
    f.write(dag_code)
    
print("\n✓ DAG guardado en: scripts/example_airflow_dag.py")

## 11. Cleanup

In [ ]:
# Cerrar sesión de Spark
# spark.stop()
# print("✓ Spark session cerrada")

## Conclusiones

### Aprendizajes Clave

1. **Integración Spark + Airflow + MLflow**
   - Airflow orquesta las tareas del pipeline
   - Spark procesa datos a escala
   - MLflow trackea experimentos y modelos

2. **Pipeline End-to-End**
   - Ingesta → Transformación → Feature Engineering → Training → Evaluation
   - Cada etapa es modular e independiente
   - Fácil de escalar y mantener

3. **Ventajas**
   - ✅ Reproducibilidad (MLflow)
   - ✅ Escalabilidad (Spark)
   - ✅ Automatización (Airflow)
   - ✅ Monitoreo centralizado

### Recursos Adicionales

- [Airflow Documentation](https://airflow.apache.org/)
- [Spark ML Guide](https://spark.apache.org/docs/latest/ml-guide.html)
- [MLflow Tracking](https://mlflow.org/docs/latest/tracking.html)

### Ejercicios Propuestos

1. Modificar el pipeline para usar otros algoritmos (GBT, SVM)
2. Implementar hyperparameter tuning con CrossValidator
3. Añadir data validation con Great Expectations
4. Crear alertas en Airflow para fallos del modelo
5. Deploy del modelo usando MLflow Model Registry